In [0]:
# Load the sensor data
sensor_df = spark.table("permanent_sensor_measurements")


In [0]:
%sql
SELECT * 
FROM permanent_sensor_measurements
WHERE 

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * 
FROM sensors_with_tract_income_brackets
WHERE confidence_level = 'High' OR confidence_level = 'Moderate'

## Create a Unified Gold View

In [0]:
%sql
CREATE OR REPLACE VIEW unified_gold_data AS 
WITH unified AS (
  SELECT 
      psm.common_key, 
      psm.location_id, 
      psm.sensors_id, 
      psm.parameter, 
      psm.units, 
      psm.value, 
      psm.datetime, 
      psm.lat, 
      psm.lon, 
      sc.GEOID, 
      sc.income_bracket, 
      sc.confidence_level
  FROM permanent_sensor_measurements psm
  LEFT JOIN sensors_with_tract_income_brackets sc
    ON psm.location_id = sc.nearest_sensor_location
)
SELECT * 
FROM unified
WHERE confidence_level IN ('High', 'Moderate');

In [0]:
%sql
WITH unified AS (
  SELECT 
      psm.common_key, 
      psm.location_id, 
      psm.sensors_id, 
      psm.parameter, 
      psm.units, 
      psm.value, 
      psm.datetime, 
      psm.lat, 
      psm.lon, 
      sc.GEOID, 
      sc.income_bracket, 
      sc.confidence_level
  FROM permanent_sensor_measurements psm
  LEFT JOIN sensors_with_tract_income_brackets sc
    ON psm.location_id = sc.nearest_sensor_location
)
SELECT * 
FROM unified
WHERE confidence_level IN ('High');

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

**Create Aggregated View for Recommendations** 

In [0]:
%sql
SELECT
  parameter
  , income_bracket
  , MAX(value) AS max_value
  , AVG(value) AS avg_value
  , COUNT(*) AS measurement_count
FROM unified_gold_data
GROUP BY parameter, income_bracket;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE OR REPLACE VIEW recommendations_summary AS 
SELECT
  GEOID
  , parameter
  , income_bracket
  , MAX(value) AS max_value
  , AVG(value) AS avg_value
  , COUNT(*) AS measurement_count
FROM unified_gold_data
GROUP BY GEOID, parameter, income_bracket;